In [1]:
import pandas as pd 
data = pd.read_csv('202101-202106(1).csv',encoding='utf-8')
data['上线月份'] = pd.to_datetime(data['上线月份'], format='%Y%m').dt.strftime('%Y-%m')

In [3]:
fur_demods_info = data[data['新旧']=='new']

In [6]:
fur_demods_info.sort_values(by=['1']

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,6
26895,19D807521G 9B9,8013,D80752-8013,new,2021-01,1,NaN,NaN,NaN,NaN,NaN
30727,11D853818 9B9,8012,D85381-8012,new,2021-04,1,10.0,NaN,1.0,1.0,NaN
30720,57D880201J 1QB,6000,D88020-6000,new,2021-03,1,1.0,2.0,1.0,1.0,NaN
30706,2Q1937090B,1000-2,193709-1000-2,new,2021-03,1,1.0,2.0,NaN,3.0,5.0
30658,3GB837206H GRU,5002,B83720-5002,new,2021-03,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50298,KIT827851,1000-1,T82785-1000-1,new,2021-04,1307,975.0,416.0,228.0,170.0,108.0
41231,1JD698451B,1000-1,D69845-1000-1,new,2021-06,1446,1280.0,1378.0,619.0,217.0,156.0
34786,101905601B,6000,190560-6000,new,2021-01,1470,1350.0,1250.0,970.0,890.0,790.0
7053,101905601B,5002,190560-5002,new,2021-01,2510,680.0,880.0,680.0,630.0,660.0


In [ ]:
1. 替代件不能当作新零件抽取，须剔除
2. 大数字，连续是1，多家经销商

In [4]:
his_demods_info[his_demods_info['相似ID']=='190560-3002']

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,6
46787,101905601F,3002,190560-3002,old,2016-07,30,10.0,NaN,NaN,NaN,NaN


In [5]:
import numpy as np
his_demods_info['to_2_rate'] = his_demods_info['2']/his_demods_info['1']
his_demods_info['to_3_rate'] = his_demods_info['3']/his_demods_info['2']
his_demods_info['to_4_rate'] = his_demods_info['4']/his_demods_info['3']
his_demods_info['to_5_rate'] = his_demods_info['5']/his_demods_info['4']
his_demods_info['to_6_rate'] = his_demods_info['6']/his_demods_info['5']
his_demods_info.fillna(0,inplace =True)
his_demods_info = his_demods_info.replace(np.inf,0)

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_60057/655279242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  his_demods_info['to_2_rate'] = his_demods_info['2']/his_demods_info['1']
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_60057/655279242.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  his_demods_info['to_3_rate'] = his_demods_info['3']/his_demods_info['2']
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_60057/655279242.py:4: SettingWithCopyWarning: 
A valu

In [6]:
his_demods_info_rate = his_demods_info[['相似ID','to_2_rate','to_3_rate','to_4_rate','to_5_rate','to_6_rate']]
his_demods_info_rate.head()

,相似ID,to_2_rate,to_3_rate,to_4_rate,to_5_rate,to_6_rate
0,D60102-3001,0.709677,0.909091,1.10,1.227273,0.481481
1,D85367-8012,2.000000,0.500000,6.00,1.000000,0.833333
2,D83969-8015,1.400000,0.571429,1.25,1.800000,0.888889
6,D85353-1000-1,0.470588,0.500000,2.75,0.454545,0.800000
7,D83305-8023,0.000000,0.000000,0.00,0.000000,2.000000


In [7]:
import pandas as pd

# 定义一个函数，用于筛选出不在15%到85%分位数范围内的值
def filter_outliers(group):
    if len(group)>=3:
        q15 = group.quantile(0.15)
        q85 = group.quantile(0.85)
        output = group[(group >= q15) & (group <= q85)]
    else:
        output =group
    return output

# 使用 groupby 分组并应用筛选函数
grouped = his_demods_info_rate.groupby('相似ID')
# 对每个列（除了 'sku' 列）应用筛选函数
filtered_df = grouped.apply(lambda group: group.iloc[:, 1:].apply(filter_outliers))

filtered_df = filtered_df.reset_index()

filtered_df

,相似ID,level_1,to_2_rate,to_3_rate,to_4_rate,to_5_rate,to_6_rate
0,00060-1000-1,10405,0.000000,0.000000,0.000000,0.000000,0.000000
1,00060-1000-1,27622,0.000000,0.000000,0.000000,0.000000,0.000000
2,00940-3006,22860,0.400000,0.000000,0.000000,0.000000,0.000000
3,00940-3006,48482,2.214286,0.548387,0.411765,0.285714,1.000000
4,01100-1000-1,909,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
45384,X01912-8022,39365,3.369748,0.082294,6.363636,0.961905,0.242574
45385,X20082-1000-3,8812,0.097166,0.083333,14.000000,3.464286,0.226804
45386,X20082-1000-3,9417,0.635417,0.213115,3.500000,2.780220,1.438735
45387,X20091-1000-3,13995,0.038217,0.000000,0.000000,0.000000,0.000000


In [8]:
output = filtered_df.groupby(['相似ID']).median().reset_index().drop('level_1',axis =1)
output

,相似ID,to_2_rate,to_3_rate,to_4_rate,to_5_rate,to_6_rate
0,00060-1000-1,0.000000,0.000000,0.000000,0.000000,0.000000
1,00940-3006,1.307143,0.274194,0.205882,0.142857,0.500000
2,01100-1000-1,0.000000,0.000000,0.000000,0.000000,0.000000
3,01508-5001,2.500000,0.575000,3.025641,0.174603,3.600000
4,02451-1000-1,0.954545,0.000000,0.000000,0.000000,0.233333
...,...,...,...,...,...,...
3146,X01911-8023,0.136820,2.000000,0.500000,0.982301,1.234588
3147,X01912-5002,0.225197,0.081746,6.191489,0.317647,0.385905
3148,X01912-8022,4.793570,0.097312,3.262617,1.331699,0.371287
3149,X20082-1000-3,0.366291,0.148224,8.750000,3.122253,0.832770


In [9]:
new_sku_demods_info = data[data['新旧']=='new']
output = pd.merge(new_sku_demods_info,output,on = ['相似ID'],how ='inner')
output.head()

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,6,to_2_rate,to_3_rate,to_4_rate,to_5_rate,to_6_rate
0,LVM765035Z2,5003,M76503-5003,new,2021-06,6,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.00,0.000000,0.000000
1,11D821022,8022,D82102-8022,new,2021-03,13,NaN,9.0,1.0,1.0,NaN,1.000000,1.071429,0.96,1.152778,0.930556
2,11D821021,8022,D82102-8022,new,2021-03,13,NaN,9.0,1.0,1.0,NaN,1.000000,1.071429,0.96,1.152778,0.930556
3,6RD612041AG,1000-2,D61204-1000-2,new,2021-01,2,3.0,3.0,8.0,11.0,7.0,0.166667,0.000000,0.00,0.000000,0.000000
4,5NG839432F 5AP,8023,G83943-8023,new,2021-04,2,NaN,1.0,5.0,2.0,4.0,2.000000,1.000000,0.00,0.000000,0.250000


In [10]:
output['2_cg'] = output['1']*output['to_2_rate']
output['3_cg'] = output['2_cg']*output['to_3_rate']
output['4_cg'] = output['3_cg']*output['to_4_rate']
output['5_cg'] = output['4_cg']*output['to_5_rate']
output['6_cg'] = output['5_cg']*output['to_6_rate']

In [11]:
for i in ['2_cg','3_cg','4_cg','5_cg','6_cg']:
    output[i] = [1 if x <0.01 else x for x in output[i]]

In [13]:
output

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,...,to_2_rate,to_3_rate,to_4_rate,to_5_rate,to_6_rate,2_cg,3_cg,4_cg,5_cg,6_cg
0,LVM765035Z2,5003,M76503-5003,new,2021-06,6,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,11D821022,8022,D82102-8022,new,2021-03,13,NaN,9.0,1.0,1.0,...,1.000000,1.071429,0.960000,1.152778,0.930556,13.000000,13.928571,13.371429,15.414286,14.343849
2,11D821021,8022,D82102-8022,new,2021-03,13,NaN,9.0,1.0,1.0,...,1.000000,1.071429,0.960000,1.152778,0.930556,13.000000,13.928571,13.371429,15.414286,14.343849
3,6RD612041AG,1000-2,D61204-1000-2,new,2021-01,2,3.0,3.0,8.0,11.0,...,0.166667,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,1.000000,1.000000,1.000000
4,5NG839432F 5AP,8023,G83943-8023,new,2021-04,2,NaN,1.0,5.0,2.0,...,2.000000,1.000000,0.000000,0.000000,0.250000,4.000000,4.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068,2G0823709,8023,082370-8023,new,2021-03,1,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.300000,1.000000,0.214286,1.000000,1.000000,1.000000,1.000000,1.000000
5069,3CG867145 TS4,1000-1,G86714-1000-1,new,2021-03,1,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5070,04E906262JP,5001,E90626-5001,new,2021-04,1,NaN,NaN,NaN,NaN,...,0.240000,0.175000,0.000000,0.000000,0.000000,0.240000,0.042000,1.000000,1.000000,1.000000
5071,18D955457B,8013,D95545-8013,new,2021-04,1,NaN,NaN,NaN,NaN,...,0.723312,0.378571,0.166667,0.333333,0.280000,0.723312,0.273825,0.045638,0.015213,1.000000


In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
def compare_two_col(r,p):
    new_df = output[[r,p]].dropna(subset=r)
    new_df['mape'] = abs(new_df[p]-new_df[r])/new_df[r]
    
    bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]
# 使用cut函数将数据分成不同区间
    new_df['mape_rate'] = pd.cut(new_df['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)
    new_df['mape_rate'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量
    print(new_df['mape'].mean())
    print(new_df['mape_rate'].value_counts(normalize=True))
    
    
print('2月份')
compare_two_col('2','2_cg')
print('3月份')

compare_two_col('3','3_cg')
print('4月份')

compare_two_col('4','4_cg')
print('5月份')

compare_two_col('5','5_cg')
print('6月份')

compare_two_col('6','6_cg')


2月份
1.2082899006683165
50%-100%    0.523001
0-10%       0.198795
>100%       0.124863
30%-50%     0.086528
10%-30%     0.066813
Name: mape_rate, dtype: float64
3月份
1.017530321337915
50%-100%    0.645833
0-10%       0.176852
>100%       0.070370
30%-50%     0.057870
10%-30%     0.049074
Name: mape_rate, dtype: float64
4月份
1.6282288138345398
50%-100%    0.641527
0-10%       0.204842
>100%       0.075885
30%-50%     0.045158
10%-30%     0.032588
Name: mape_rate, dtype: float64
5月份
6.851301115212573
50%-100%    0.642891
0-10%       0.207755
>100%       0.078506
10%-30%     0.038775
30%-50%     0.032073
Name: mape_rate, dtype: float64
6月份
2.046316972606514
50%-100%    0.648045
0-10%       0.209032
>100%       0.078212
30%-50%     0.034451
10%-30%     0.030261
Name: mape_rate, dtype: float64
